In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# mount
path_to_project = "/content/drive/MyDrive/UDEM/ift6010/Project6010/ift6010-h21-team1" #@param string
%cd $path_to_project
!git pull
!bash setup.sh

/content/drive/.shortcut-targets-by-id/1aEENb1YnxFjQj4ST_k0N0dicvLsCGYTC/Project6010/ift6010-h21-team1
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/pbreau3/ift6010-h21-team1
   5000843..72446e0  main       -> origin/main
Updating 5000843..72446e0
Fast-forward
 distance.py             | 11 ++++++-----
 distance_python2.py     | 11 ++++++-----
 distance_python2.py.bak | 11 ++++++-----
 3 files changed, 18 insertions(+), 15 deletions(-)


In [3]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	__pycache__/
	generate_kenlm_python2.pyc
	item.txt
	item3.txt
	kenlm.so
	kenlm_cpp.pyc
	sample-1.mid
	sample-2.mid
	sample-3.mid
	sample-4.mid
	twinkle_complete.mid
	vocab.pyc
	vocab_python2.pyc

nothing added to commit but untracked files present (use "git add" to track)


# KenLM
For the n-gram construction, we will use kenlm (https://github.com/kpu/kenlm). Kenlm uses Knesey-Ney smoothing to deal with low-frequency n-grams. 

You need to build it from source code using CMake.

## Compilation
Compile outside of the repo. Modify the `kenlm.sh` script aliases to match the binaries.

In [4]:
%cd /
!wget -O - https://kheafield.com/code/kenlm.tar.gz |tar xz
%mkdir kenlm/build
%cd kenlm/build
!cmake -DENABLE_PYTHON=ON -DKENLM_MAX_ORDER=20 -DPYTHON_VERSION_STRING=3.7 ..
!make -j2 install
%cd $path_to_project

/
--2021-04-25 15:08:53--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491090 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 479.58K  --.-KB/s    in 0.03s   

2021-04-25 15:08:53 (18.3 MB/s) - written to stdout [491090/491090]

/kenlm/build
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detec

Python extension compilation (C++) should be made automatically by cmake

In [5]:
%cp /usr/local/lib/python2.7/site-packages/kenlm.so $path_to_project/

# Training
Large n-gram order require recompilation with `cmake -DKENLM_MAX_ORDER=10 ..`
You may have to rewire aliases in the `kenlm.sh` script.

In [ ]:
# group all training files into one
!bash group.sh "data/JSB Chorales/train/text" musicautobot

In [ ]:
# bigram
!bash kenlm.sh 20 model/group/musicautobot

## Evaluation
Model perplexity is the amount of uncertainty the model has when given an input. High perplexity means the model is not confident about its predictions.

In [ ]:
!python2 evaluate_python2.py

# Generation
Create a vocabulary

In [ ]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/pbreau3/ift6010-h21-team1
   bd241a0..5000843  main       -> origin/main
Updating bd241a0..5000843
Fast-forward
 distance.py             | 3 +++
 distance_python2.py     | 3 +++
 distance_python2.py.bak | 3 +++
 3 files changed, 9 insertions(+)


In [ ]:
# display vocab, strange but requires using a scripting command
!python -c "from vocab import get_vocab; print(get_vocab(\"data/JSB Chorales/voc_musicautobot.voc\"))"

['n74', 'n75', 'n76', 'n77', 'n70', 'n71', 'n72', 'n66', 'n56', 'n57', 'n54', 'n55', 'n78', 'n79', 'n50', 'n51', 'n88', 'd30', 'd9', 'd6', 'd4', 'd5', 'n73', 'd14', 'd15', 'n49', 'd10', 'n58', 'n96', 'd39', 'n92', 'n93', 'n90', 'n91', 'd18', 'd19', 'n94', 'n95', 'xxsep', 'd38', 'n65', 'd53', 'd96', 'd87', 'd57', 'n67', 'n48', 'xxpad', 'n64', 'n63', 'n62', 'n61', 'n60', 'n52', 'n43', 'd91', 'n69', 'n68', 'n46', 'd33', 'd29', 'n84', 'n87', 'n86', 'n81', 'n80', 'n83', 'n82', 'd20', 'n89', 'd24', 'n59', 'd48', 'n85', 'n53', 'd43', 'd67', 'xxbos']


Generate

In [ ]:
!python2 generate_kenlm_python2.py "model/kenlm/order16.bin" \
  "data/JSB Chorales/voc_musicautobot.voc" \
  "xxbos xxpad n91 d5 n84 d9 n75 d5 n60 d5 xxsep"

model/kenlm/order16.bin
Premise: xxbos xxpad n91 d5 n84 d9 n75 d5 n60 d5 xxsep
d6 n92 d5 n87 d9 n77 d5 xxsep d4 n96 d5 n80 d9 n68 d5 n53 d9 xxsep d6 n79 d53 n67 d5 xxsep d4 n83 d5 n67 d24 n55 d5 xxsep d4 n72 d19 n64 d19 n48 d15 xxsep d14 n60 d5 xxsep d6 n76 d9 n69 d19 n57 d19 xxsep d14 n84 d5 n76 d5 n69 d24 xxsep d4 n57 d5 xxsep d6 n86 d15 n77 d15 n62 d15 n58 d9 xxsep d10 n56 d5 xxsep d4 n75 d9 n66 d5 n57 d5 xxsep d6 n70 d48 n62 d15 n58 d9 xxsep d10 n56 d5 xxsep d4 n75 d9 n66 d5 n57 d5 xxsep d6 n70 d48 n62 d15 n58 d9 xxsep d10 n56 d5 xxsep d4 n75 d9 n66 d5 n57 d5 xxsep d6 n70 d48 n62 d15 n58 d9 xxsep d10 n56 d5 xxsep d4 n75 d9 n66 d5 n57 d5 xxsep d6 n70 d48 n62 d15 n58 d9 xxsep d10 n56 d5 xxsep d4 n75 d9 n66 d5 n57 d5 xxsep d6 n70 d48 n62 d15 n58 d9 xxsep d10 n56 d5 xxsep d4 n75 d9 n66 d5 n57 d5 xxsep d6 n70 d48 n62 d15 n58 d9 xxsep d10 n56 d5 xxsep d4 n75


## Evaluation

In [33]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/pbreau3/ift6010-h21-team1
   e4a6883..76eee21  main       -> origin/main
Updating e4a6883..76eee21
Fast-forward
 distance.sh         | 14 ++++++++------
 distance_python2.py |  1 +
 2 files changed, 9 insertions(+), 6 deletions(-)


In [ ]:
import os
for file in os.listdir("data/JSB Chorales/test/text/"):
  !python distance.py "data/JSB Chorales/test/text/"$file "data/JSB Chorales/premiseless_order16.txt" --levenshtein

In [30]:
!bash distance.sh

8007 3268.002753976808 log/distances_order10.log
7987.987012987013 3261.5234392176244 log/distances_order11.log
7982.506493506494 3263.585853270572 log/distances_order12.log
7983.441558441558 3262.3720321485266 log/distances_order13.log
7982.974025974026 3263.9093687995573 log/distances_order14.log
7984.337662337663 3263.0658678868476 log/distances_order15.log
7986.623376623376 3262.658207654099 log/distances_order16.log
7986.636363636364 3262.3426153169316 log/distances_order17.log
7978.883116883117 3259.581535840169 log/distances_order18.log
7982.857142857143 3259.1553059451885 log/distances_order19.log
7984.792207792208 3258.8188259651374 log/distances_order20.log
15273.61038961039 5909.008132450346 log/distances_order2.log
8007.220779220779 3329.6725997713056 log/distances_order3.log
8120.025974025974 3279.8298093443573 log/distances_order4.log
7962.493506493506 3182.1586955670628 log/distances_order5.log
8098.467532467533 3243.3037679454 log/distances_order6.log
8066.558441558442 